# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time

# Import API key
from api_keys_Yvonne import g_key
# Access maps with unique API key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#from weatherpy
df = pd.read_csv('yvonne_data.csv')
df

,City,Cloudiness,Country,Date,Humidity %,Latitude,Longitude,Temperature (F),Wind_Speed,Pressure
0,Norman Wells,75.0,CA,1.644134e+09,69.0,65.2820,-126.8329,-18.36,2.30,1002.0
1,Ushuaia,0.0,AR,1.644134e+09,70.0,-54.8000,-68.3000,44.26,5.75,1012.0
2,Chui,90.0,UY,1.644134e+09,89.0,-33.6971,-53.4616,64.04,8.50,1008.0
3,Atuona,3.0,PF,1.644134e+09,77.0,-9.8000,-139.0333,78.98,22.88,1014.0
4,Busselton,100.0,AU,1.644134e+09,85.0,-33.6500,115.3333,64.74,24.32,1001.0
...,...,...,...,...,...,...,...,...,...,...
571,Praia,20.0,CV,1.644135e+09,82.0,14.9215,-23.5087,66.74,13.80,1014.0
572,Nelson Bay,97.0,AU,1.644135e+09,66.0,-32.7167,152.1500,72.05,3.00,1019.0
573,Hamīrpur,0.0,IN,1.644135e+09,34.0,25.9500,80.1500,70.77,6.35,1015.0
574,Verkhnevilyuysk,61.0,RU,1.644135e+09,100.0,63.4453,120.3167,-13.86,3.87,1022.0


In [3]:
df2 = df.dropna().reset_index(drop=True)

In [4]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 574 entries, 0 to 573
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   City             574 non-null    object 
 1   Cloudiness       574 non-null    float64
 2   Country          574 non-null    object 
 3   Date             574 non-null    float64
 4   Humidity %       574 non-null    float64
 5   Latitude         574 non-null    float64
 6   Longitude        574 non-null    float64
 7   Temperature (F)  574 non-null    float64
 8   Wind_Speed       574 non-null    float64
 9   Pressure         574 non-null    float64
dtypes: float64(8), object(2)
memory usage: 45.0+ KB


In [5]:
df2.columns

Index(['City', 'Cloudiness', 'Country', 'Date', 'Humidity %', 'Latitude',
       'Longitude', 'Temperature (F)', 'Wind_Speed', 'Pressure'],
      dtype='object')

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
# Humidity Heatmap
locations = df2.loc[:, ["Latitude", "Longitude"]]

fig = gmaps.figure()

#add heatmap layer
heatmap_layer = gmaps.heatmap_layer(
    locations, weights=df2['Humidity %'],
    max_intensity=75, point_radius=10)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# save image


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
#Narrow down the DataFrame to find your ideal weather condition.

#A max temperature lower than 80 degrees but higher than 70.
#Wind speed less than 10 mph.
#Zero cloudiness.
#Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

In [9]:
df2.columns

Index(['City', 'Cloudiness', 'Country', 'Date', 'Humidity %', 'Latitude',
       'Longitude', 'Temperature (F)', 'Wind_Speed', 'Pressure'],
      dtype='object')

In [10]:
#use mask with criteria from above
mask = (df2["Temperature (F)"] < 80) & (df2["Temperature (F)"] > 70) & (df2["Wind_Speed"] < 10) & (df2["Cloudiness"] == 0)
df2.loc[mask]

,City,Cloudiness,Country,Date,Humidity %,Latitude,Longitude,Temperature (F),Wind_Speed,Pressure
14,Vila Velha,0.0,BR,1.644134e+09,88.0,-20.3297,-40.2925,74.95,5.75,1013.0
45,Shwebo,0.0,MM,1.644134e+09,30.0,22.5667,95.7000,79.63,7.74,1010.0
104,Acapulco de Juárez,0.0,MX,1.644134e+09,71.0,16.8634,-99.8901,73.40,4.88,1014.0
171,São Filipe,0.0,CV,1.644134e+09,69.0,14.8961,-24.4956,72.34,8.68,1013.0
216,Burnie,0.0,AU,1.644134e+09,52.0,-41.0667,145.9167,70.95,6.51,1028.0
234,Bhitarwār,0.0,IN,1.644135e+09,25.0,25.8000,78.1167,73.62,2.10,1015.0
303,Fort Abbas,0.0,PK,1.644135e+09,22.0,29.1925,72.8536,73.71,8.57,1013.0
507,Porbandar,0.0,IN,1.644135e+09,45.0,21.6422,69.6093,74.59,6.78,1014.0
571,Hamīrpur,0.0,IN,1.644135e+09,34.0,25.9500,80.1500,70.77,6.35,1015.0


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [11]:
vac_cities = df2.loc[mask]
vac_cities

,City,Cloudiness,Country,Date,Humidity %,Latitude,Longitude,Temperature (F),Wind_Speed,Pressure
14,Vila Velha,0.0,BR,1.644134e+09,88.0,-20.3297,-40.2925,74.95,5.75,1013.0
45,Shwebo,0.0,MM,1.644134e+09,30.0,22.5667,95.7000,79.63,7.74,1010.0
104,Acapulco de Juárez,0.0,MX,1.644134e+09,71.0,16.8634,-99.8901,73.40,4.88,1014.0
171,São Filipe,0.0,CV,1.644134e+09,69.0,14.8961,-24.4956,72.34,8.68,1013.0
216,Burnie,0.0,AU,1.644134e+09,52.0,-41.0667,145.9167,70.95,6.51,1028.0
234,Bhitarwār,0.0,IN,1.644135e+09,25.0,25.8000,78.1167,73.62,2.10,1015.0
303,Fort Abbas,0.0,PK,1.644135e+09,22.0,29.1925,72.8536,73.71,8.57,1013.0
507,Porbandar,0.0,IN,1.644135e+09,45.0,21.6422,69.6093,74.59,6.78,1014.0
571,Hamīrpur,0.0,IN,1.644135e+09,34.0,25.9500,80.1500,70.77,6.35,1015.0


In [12]:
#Day6-Python-APIs-3-Activities-05-Stu_Google_Complex, booth
# geocoordinates test from a city above
target_coordinates = f"{22.0752	}, {-159.3190}"

target_radius = 5000
target_type = "lodging"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
response = requests.get(base_url, params=params)

places_data = response.json()

print(places_data["results"][0]["name"])
print(places_data["results"][0]["vicinity"])
print(places_data["results"][0]["rating"])

Sheraton Kauai Coconut Beach Resort
650 Aleka Loop, Kapaʻa
4.1


In [13]:
# exactly like in #Day6-Python-APIs-3-Activities-05-Stu_Google_Complex solved
hotel_df= vac_cities

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key}

# 
for index, row in vac_cities.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    
    places_data = response.json()
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = places_data["results"][0]["name"]
       
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

C:\Users\yvonne\anaconda3\lib\site-packages\pandas\core\indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
C:\Users\yvonne\anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [14]:
hotel_df

,City,Cloudiness,Country,Date,Humidity %,Latitude,Longitude,Temperature (F),Wind_Speed,Pressure,Hotel Name
14,Vila Velha,0.0,BR,1.644134e+09,88.0,-20.3297,-40.2925,74.95,5.75,1013.0,Quality Suites Vila Velha
45,Shwebo,0.0,MM,1.644134e+09,30.0,22.5667,95.7000,79.63,7.74,1010.0,MOON SHINE Guest House
104,Acapulco de Juárez,0.0,MX,1.644134e+09,71.0,16.8634,-99.8901,73.40,4.88,1014.0,HS Hotsson Smart Acapulco
171,São Filipe,0.0,CV,1.644134e+09,69.0,14.8961,-24.4956,72.34,8.68,1013.0,Hotel Xaguate
216,Burnie,0.0,AU,1.644134e+09,52.0,-41.0667,145.9167,70.95,6.51,1028.0,Beachfront Voyager Motor Inn
234,Bhitarwār,0.0,IN,1.644135e+09,25.0,25.8000,78.1167,73.62,2.10,1015.0,H.G.Batham Home
303,Fort Abbas,0.0,PK,1.644135e+09,22.0,29.1925,72.8536,73.71,8.57,1013.0,AsAd's House
507,Porbandar,0.0,IN,1.644135e+09,45.0,21.6422,69.6093,74.59,6.78,1014.0,Lords Inn Porbandar
571,Hamīrpur,0.0,IN,1.644135e+09,34.0,25.9500,80.1500,70.77,6.35,1015.0,Bundalkhand kurara transport company


In [15]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations2 = hotel_df[["Latitude", "Longitude"]]

In [16]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations2, info_box_content=hotel_info)

# Display figure
fig = gmaps.figure()
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [17]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations2, info_box_content=hotel_info)
heatmap_layer = gmaps.heatmap_layer(locations, weights=df2['Humidity %'], max_intensity=75, point_radius=10)


# Display figure
fig = gmaps.figure()
#with both layers
fig.add_layer(marker_layer)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [18]:
#We have a couple of hotels to choose from that are within our asks of weather.